Nama : Drajat Fransetio
NIM : STI202303344
Kelas : C4.1
Matkul : Kecerdasan Buatan

In [1]:
# Langkah 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Langkah 2: Install package yang diperlukan
!pip install --upgrade --quiet pip
!pip install numpy==1.23.5 tensorflow==2.12.0 opencv-python

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Langkah 3: Import library yang diperlukan
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode
import base64

print(f"TensorFlow: {tf.__version__}")  # Harus 2.12.0
print(f"NumPy: {np.__version__}")       # Harus 1.23.5

TensorFlow: 2.12.0
NumPy: 1.23.5


In [3]:
# Langkah 4: Load model dan labels
MODEL_PATH = '/content/drive/MyDrive/AI/Dataset/MyDataset/Model2/keras_model.h5'
LABEL_PATH = '/content/drive/MyDrive/AI/Dataset/MyDataset/Model2/labels.txt'

# Load model
model = load_model(MODEL_PATH, compile=False)

# Load labels
with open(LABEL_PATH, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
!cat '/content/drive/MyDrive/AI/Dataset/MyDataset/Model2/labels.txt'

0 Class 1: Satu Gelas Kopi
1 Class 2: Dompet
2 Class 3: Korek Api
3 Class 4: TWS
4 Class 5: Handhpone
5 Class 6: Kacamata


In [4]:
#Langkah 5
# Fungsi prediksi benda
def predict_from_frame(frame):
    # Preprocessing gambar
    resized = cv2.resize(frame, (224, 224))  # Sesuaikan dengan input model
    normalized = resized.astype('float32') / 127.5 - 1  # Normalisasi untuk Teachable Machine
    input_array = np.expand_dims(normalized, axis=0)

    # Lakukan prediksi
    predictions = model.predict(input_array)

    # Ambil label dan confidence
    class_id = np.argmax(predictions[0])
    confidence = predictions[0][class_id]
    label = labels[class_id]

    return label, confidence

# Fungsi untuk menampilkan UI
def show_camera_ui():
    display(HTML("""
    <div style="text-align:center">
        <button id="capture-btn" style="
            padding:12px 24px;
            background:#4CAF50;
            color:white;
            border:none;
            border-radius:4px;
            font-size:16px;
            margin:10px;">
            CAPTURE
        </button>
        <div id="status" style="margin:10px;"></div>
    </div>
    """))

# Fungsi take_photo
def take_photo():
    js = Javascript("""
    async function capture() {
        const status = document.getElementById('status');
        status.innerHTML = 'Mengakses kamera...';

        try {
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            status.innerHTML = 'Kamera siap - Klik CAPTURE';

            const dataUrl = await new Promise((resolve) => {
                document.getElementById('capture-btn').onclick = () => {
                    const canvas = document.createElement('canvas');
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    stream.getVideoTracks()[0].stop();
                    video.remove();
                    status.innerHTML = 'Mengambil gambar...';
                    resolve(canvas.toDataURL('image/jpeg'));
                };
            });

            return dataUrl;
        } catch (error) {
            status.innerHTML = 'Error: ' + error.message;
            return null;
        }
    }
    """)

    display(js)
    try:
        result = eval_js("capture()")
        if not result:
            print("⚠️ Gagal mengambil gambar dari webcam")
            return None
        return result
    except Exception as e:
        print(f"❌ Error JavaScript: {str(e)}")
        return None

# Fungsi utama
def run_detection():
    print("🟢 Sistem siap")
    show_camera_ui()

    while True:
        try:
            # Ambil foto
            data_url = take_photo()
            if not data_url:
                print("⚠️ Tidak ada data gambar, coba lagi")
                continue

            # Ekstrak data gambar
            header, encoded = data_url.split(",", 1)
            binary = b64decode(encoded)

            # Konversi ke OpenCV format
            nparr = np.frombuffer(binary, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            if frame is None:
                print("⚠️ Gagal memproses gambar")
                continue

            # Prediksi menggunakan fungsi predict_from_frame yang sebenarnya
            label, conf = predict_from_frame(frame)

            # Konversi frame ke RGB untuk ditampilkan
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Tambahkan teks hasil prediksi
            frame_rgb = cv2.putText(frame_rgb, f"{label} ({conf*100:.1f}%)",
                                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                                   1, (0, 255, 0), 2)

            # Encode gambar untuk ditampilkan
            _, img_encoded = cv2.imencode('.jpg', frame_rgb)
            display(HTML(f"""
                <div style="text-align:center;margin-top:20px">
                    <img src="data:image/jpeg;base64,{base64.b64encode(img_encoded).decode()}"
                         style="max-width:500px;border:2px solid #4CAF50">
                    <p style="margin-top:10px;font-size:16px">
                        Hasil: <strong>{label}</strong> |
                        Akurasi: <strong>{conf*100:.1f}%</strong>
                    </p>
                </div>
            """))

            if input("Tekan Enter untuk lanjut atau 'q'+Enter untuk keluar: ").lower() == 'q':
                print("🔴 Program dihentikan")
                break

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            break

# Jalankan program
run_detection()

🟢 Sistem siap


<IPython.core.display.Javascript object>

⚠️ Gagal mengambil gambar dari webcam
⚠️ Tidak ada data gambar, coba lagi


<IPython.core.display.Javascript object>

⚠️ Gagal mengambil gambar dari webcam
⚠️ Tidak ada data gambar, coba lagi


...

⚠️ Gagal mengambil gambar dari webcam
⚠️ Tidak ada data gambar, coba lagi


<IPython.core.display.Javascript object>

⚠️ Gagal mengambil gambar dari webcam
⚠️ Tidak ada data gambar, coba lagi


<IPython.core.display.Javascript object>

KeyboardInterrupt: 